<a href="https://colab.research.google.com/github/Mp-Pro/Movie-recommendation/blob/main/Movierecomm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CONTENT BASED RECOMMENDATION -- cosine similarity

# Cleaning and filtering the datasets

In [1]:
# import tensorflow as tf
# tf.test.gpu_device_name()

In [2]:
# import pandas as pd
# credits = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Movie Recommendation System/datasets/credits.csv")
# links = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Movie Recommendation System/datasets/links.csv")
# movies_metadata = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Movie Recommendation System/datasets/movies_metadata.csv", low_memory=False)
# ratings = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Movie Recommendation System/datasets/ratings.csv")
# keywords = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Movie Recommendation System/datasets/keywords.csv")

In [3]:
# movies_metadata.drop(['budget','original_language','video','runtime','status','video','production_countries','release_date','revenue','spoken_languages'],axis = 1,inplace=True)

In [4]:
# movies_metadata.head()

In [5]:
# ratings.drop(['timestamp'],axis=1,inplace=True)

In [6]:
# ratings.rename(columns = {'movieId':'id'},inplace=True)

In [7]:
# keywords['id'].dtype

In [8]:
# credits = credits.astype({'id':'str'})
# movies_metadata = movies_metadata.astype({'id':'str'})
# ratings['id'] = ratings['id'].astype('str')
# keywords['id'] = keywords['id'].astype('str')

In [9]:
# merged_data = movies_metadata.merge(credits,on='id')

In [10]:
# print(merged_data.shape)

In [11]:
# print(merged_data['id'].dtype)
# print(ratings['id'].dtype)

In [12]:
# merged_data = merged_data.merge(ratings,on='id')

In [13]:
# merged_data = merged_data.merge(keywords,on='id')
# print(merged_data['keywords'])

In [14]:
# print(credits.shape)
# print(ratings.shape)
# print(movies_metadata.shape)
# print(keywords.shape)
# print(merged_data.shape)

In [15]:
# merged_data

In [16]:
# merged_data.drop_duplicates(subset=['id']).shape

In [17]:
# movies_metadata.drop_duplicates(subset=['id'],inplace=True)

In [18]:
# ratings.drop_duplicates(subset=['id'],inplace=True)

In [19]:
# keywords.drop_duplicates(subset=['id'],inplace=True)

In [20]:
# credits.drop_duplicates(subset=['id'],inplace=True)

In [21]:
# merged_data = pd.merge(movies_metadata,keywords,on='id')

In [22]:
# merged_data = pd.merge(merged_data,credits,on='id')

In [23]:
# merged_data.shape

In [24]:
# merged_data.isnull().sum()

In [25]:
# merged_data.drop(['belongs_to_collection','homepage','tagline','original_language','release_date','revenue','runtime','spoken_languages','status'],axis=1,inplace=True)

In [26]:
# merged_data.isnull().sum()

In [27]:
# merged_data.drop(['production_countries','video','adult','budget'],axis=1,inplace=True)

In [28]:
# merged_data.dropna(inplace=True)

In [29]:
# merged_data.isnull().sum()

In [30]:
# merged_data.shape

In [31]:
# merged_data

In [32]:
# merged_data.drop_duplicates(subset=['id'],inplace = True)

In [33]:
# merged_data.shape

In [34]:
# merged_data.to_csv('merged_file.csv')
# from google.colab import files
# files.download("merged_file.csv")

In [35]:
# merged_data.shape

In [36]:
# [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]

# to ['Animation','Comedy','Family']

# Working on the Merged Csv

In [37]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [38]:
import pandas as pd
merged_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Movie Recommendation System/datasets/merged_file.csv')

In [39]:
# convert the string of list into lists
import ast
# ast.literal_eval()

In [40]:
def convert(obj):
  L = []
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [41]:
merged_data['genres'] = merged_data['genres'].apply(convert)

In [42]:
merged_data['keywords'] = merged_data['keywords'].apply(convert)

In [43]:
def convert3(obj):
  L = []
  ctr = 0
  for i in ast.literal_eval(obj):
    if ctr==3:
      break
    L.append(i['name'])
    ctr+=1
  return L

In [44]:
merged_data['cast'] = merged_data['cast'].apply(convert3)

In [45]:
def fetch_director(obj):
  L = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director' :
      L.append(i['name'])
      break
  return L

In [46]:
merged_data['crew'] = merged_data['crew'].apply(fetch_director)

In [47]:
merged_data['crew']

0           [John Lasseter]
1            [Joe Johnston]
2           [Howard Deutch]
3         [Forest Whitaker]
4           [Charles Shyer]
                ...        
44116    [Hamid Nematollah]
44117            [Lav Diaz]
44118      [Mark L. Lester]
44119    [Yakov Protazanov]
44120       [Daisy Asquith]
Name: crew, Length: 44121, dtype: object

In [48]:
# merged_data['overview'] = merged_data['overview'].apply(lambda x:x.split())

In [49]:
merged_data['genres'] = merged_data['genres'].apply(lambda x:[i.replace(" ","") for i in x])
merged_data['keywords'] = merged_data['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
merged_data['cast'] = merged_data['cast'].apply(lambda x:[i.replace(" ","") for i in x])
merged_data['crew'] = merged_data['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [50]:
# merged_data['tags'] = merged_data['overview'] + merged_data['genres'] + merged_data['cast'] + merged_data['crew']

In [51]:
merged_data['tags'] = merged_data['genres'] + merged_data['cast'] + merged_data['crew']

In [52]:
newdf = merged_data[['id','title','tags']]
newdf

,id,title,tags
0,862,Toy Story,"[Animation, Comedy, Family, TomHanks, TimAllen..."
1,8844,Jumanji,"[Adventure, Fantasy, Family, RobinWilliams, Jo..."
2,15602,Grumpier Old Men,"[Romance, Comedy, WalterMatthau, JackLemmon, A..."
3,31357,Waiting to Exhale,"[Comedy, Drama, Romance, WhitneyHouston, Angel..."
4,11862,Father of the Bride Part II,"[Comedy, SteveMartin, DianeKeaton, MartinShort..."
...,...,...,...
44116,439050,Subdue,"[Drama, Family, LeilaHatami, KouroshTahami, El..."
44117,111109,Century of Birthing,"[Drama, AngelAquino, PerryDizon, HazelOrencio,..."
44118,67758,Betrayal,"[Action, Drama, Thriller, ErikaEleniak, AdamBa..."
44119,227506,Satan Triumphant,"[IwanMosschuchin, NathalieLissenko, PavelPavlo..."


In [53]:
newdf ['tags'] = newdf ['tags'].apply(lambda x:" ".join(x))

<ipython-input-53-8f99a6b03e24>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf ['tags'] = newdf ['tags'].apply(lambda x:" ".join(x))


In [54]:
newdf ['tags'] = newdf ['tags'].apply(lambda x:x.lower())

<ipython-input-54-92bd0824706c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf ['tags'] = newdf ['tags'].apply(lambda x:x.lower())


In [55]:
import nltk
import numpy as np
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


In [56]:
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  
  return " ".join(y)

In [57]:
newdf['tags'] = newdf['tags'].apply(stem)

<ipython-input-57-53d0dca3d307>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['tags'] = newdf['tags'].apply(stem)


In [58]:

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000 ,stop_words='english',lowercase=True)

In [59]:
vectors = cv.fit_transform(newdf['tags']).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [60]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [61]:
vectors.shape

(44121, 10000)

In [62]:
newdf.shape

(44121, 3)

In [63]:
# from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Change chunk_size to control resource consumption and speed
# Higher chunk_size means more memory/RAM needed but also faster 
cosine_similarity_chunk=np.empty((0,vectors.shape[0]))
chunk_size = 500
matrix_len = vectors.shape[0] # Not sparse numpy.ndarray

def similarity_cosine_by_chunk(start, end):
    if end > matrix_len:
        end = matrix_len
    return cosine_similarity(X=vectors[start:end], Y=vectors) # scikit-learn function

for chunk_start in range(0, matrix_len, chunk_size):
    cosine_similarity_chunk = np.concatenate((cosine_similarity_chunk, similarity_cosine_by_chunk(chunk_start, chunk_start+chunk_size)))
    # Handle cosine_similarity_chunk  ( Write it to file_timestamp and close the file )
    # Do not open the same file again or you may end up with out of memory after few chunks 

In [ ]:
cosine_similarity_chunk.shape

In [ ]:
# similarity = cosine_similarity(vectors[1:20000])

In [ ]:
def recommend(movie):
  movie_index = newdf[newdf['title'].apply(lambda x: x.lower()) == movie.lower()].index[0]
  distances = cosine_similarity_chunk[movie_index]
  movies_list = sorted(list(enumerate(distances)),reverse=True,key = lambda x:x[1])[0:10]
  for i in movies_list:
    print(newdf.iloc[i[0]].title)

In [ ]:
newdf['tags'][0]

In [ ]:
# movie = "thor"
# movie_index = newdf[newdf['title'].apply(lambda x: x.lower()) == movie.lower()].index[0]
# print(movie_index)

In [ ]:
cosine_similarity_chunk.shape

In [ ]:
recommend("toy story")